In [55]:
# !git clone https://github.com/muxspace/facial_expressions.git

In [1]:
import csv
data = {}
with open('content/facial_expressions/data/legend.csv') as f:
    reader = csv.reader(f)
    next(reader)
    for row in reader:
        key = row[2].lower()
        if key in data:
            data[key].append(row[1])
        else:
            data[key] = [row[1]]

In [2]:
emotion_list = list(data.keys())
emotion_list

['anger',
 'surprise',
 'disgust',
 'fear',
 'neutral',
 'happiness',
 'sadness',
 'contempt']

In [3]:
import os

os.mkdir('content/master_data')
os.mkdir('content/master_data/training')
os.mkdir('content/master_data/testing')

In [4]:
for emotion in emotion_list:
    os.mkdir(os.path.join('content/master_data/training/', emotion))
    os.mkdir(os.path.join('content/master_data/testing/', emotion))

In [5]:
from shutil import copyfile
split_size = 0.8

for emotion, images in data.items():
    train_size = int(split_size*len(images))
    train_images = images[:train_size]
    test_images = images[train_size:]
    for image in train_images:
        source = os.path.join('content/facial_expressions/images', image)
        dest = os.path.join('content/master_data/training', emotion, image)
        copyfile(source, dest)
    for image in test_images:
        source = os.path.join('content/facial_expressions/images', image)
        dest = os.path.join('content/master_data/testing', emotion, image)
        copyfile(source, dest)

In [6]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten

In [7]:
model = tf.keras.models.Sequential([
      Conv2D(16, (3,3), activation='relu', input_shape = (100, 100, 3)),
      MaxPooling2D(2, 2),
      Conv2D(32, (3,3), activation='relu'),
      MaxPooling2D(2, 2),
      Conv2D(64, (3,3), activation='relu'),
      MaxPooling2D(2, 2),
      Flatten(),
      Dense(1024, activation='relu'),
      Dense(8, activation='softmax')
])
model.compile(optimizer = Adam(lr = 0.01), loss = 'categorical_crossentropy', metrics = ['acc'])
model.summary()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 98, 98, 16)        448       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 49, 49, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 47, 47, 32)        4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 23, 23, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 21, 21, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 10, 10, 64)        0         
_________________________________________

In [8]:
train_dir = 'content/master_data/training'
test_dir = 'content/master_data/testing'

train_datagen = ImageDataGenerator(rescale = 1.0/255)
train_generator = train_datagen.flow_from_directory(
                                                    train_dir,
                                                    target_size = (100, 100),
                                                    class_mode = 'categorical',
                                                    batch_size = 128
                                                  )

test_datagen = ImageDataGenerator(rescale = 1.0/255)
test_generator = test_datagen.flow_from_directory(
                                                    test_dir,
                                                    target_size = (100, 100),
                                                    class_mode = 'categorical',
                                                    batch_size = 128
                                                  )

Found 10941 images belonging to 8 classes.
Found 2742 images belonging to 8 classes.


In [9]:
es = EarlyStopping(monitor='val_acc', patience = 2, min_delta=0.01)

In [10]:
model.fit_generator(train_generator,
                    epochs = 10,
                    verbose = 1,
                    validation_data = test_generator,
                    callbacks = [es])

2022-05-04 17:13:42.741175: I tensorflow/core/platform/cpu_feature_guard.cc:145] This TensorFlow binary is optimized with Intel(R) MKL-DNN to use the following CPU instructions in performance critical operations:  SSE4.1 SSE4.2
To enable them in non-MKL-DNN operations, rebuild TensorFlow with the appropriate compiler flags.
2022-05-04 17:13:42.742320: I tensorflow/core/common_runtime/process_util.cc:115] Creating new thread pool with default inter op setting: 10. Tune using inter_op_parallelism_threads for best performance.


Epoch 1/10
86/86 [==============================] - 329s 4s/step - loss: 1.2546 - acc: 0.4929 - val_loss: 1.0256 - val_acc: 0.5715
Epoch 2/10
86/86 [==============================] - 257s 3s/step - loss: 0.9005 - acc: 0.6537 - val_loss: 0.9203 - val_acc: 0.6506
Epoch 3/10
86/86 [==============================] - 255s 3s/step - loss: 0.7800 - acc: 0.7267 - val_loss: 0.9947 - val_acc: 0.6411
Epoch 4/10
86/86 [==============================] - 223s 3s/step - loss: 0.7123 - acc: 0.7529 - val_loss: 0.9137 - val_acc: 0.6721
Epoch 5/10
86/86 [==============================] - 227s 3s/step - loss: 0.6276 - acc: 0.7835 - val_loss: 0.9114 - val_acc: 0.6823
Epoch 6/10
86/86 [==============================] - 244s 3s/step - loss: 0.5783 - acc: 0.7984 - val_loss: 1.7210 - val_acc: 0.6196
Epoch 7/10
86/86 [==============================] - 200s 2s/step - loss: 0.5731 - acc: 0.8003 - val_loss: 1.0674 - val_acc: 0.6499
